In [1]:
import torch
from torch import nn
from torch.nn import init
import torch.optim as optim
import torch.utils.data as Data
import numpy as np
import random

In [2]:
# 生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
# 读取数据
batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.9105, -1.4444],
        [-1.3814,  0.1128],
        [ 0.4903, -0.4180],
        [ 0.3948, -0.1033],
        [ 0.0717, -0.1322],
        [-0.3644,  0.1312],
        [ 0.9015, -0.7007],
        [-1.7804, -1.5069],
        [-2.0331,  0.0982],
        [-0.6443,  0.1427]]) tensor([ 7.2874,  1.0476,  6.6009,  5.3322,  4.7912,  3.0228,  8.3861,  5.7467,
        -0.2069,  2.4081])


In [4]:
# 定义模型
class LinearNet(nn.Module):
    def __init__(self, num_features):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_features, 1) #进2出1
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

# 通过net.parameters()查看可学习参数
for param in net.parameters():
    print(param)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[-0.0796, -0.6118]], requires_grad=True)
Parameter containing:
tensor([-0.2267], requires_grad=True)


In [ ]:
# 使用容器类nn.Sequential
# 写法1
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    )

In [29]:
# 写法2
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

In [11]:
# 初始化模型参数
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)
# net.linear.bias.data.fill_(0)

for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0031,  0.0093]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [6]:
# 定义损失函数
loss = nn.MSELoss()

In [7]:
# 定义优化算法
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
# 为不同子网络设置不同学习率
optimizer = optim.SGD([{'params': net.subnet1.parameters()}
                       {'params': net.subnet2.parameters(), 'lr': 0.01}
                      ], lr = 0.03)

In [ ]:
# 调整损失函数，也可以直接建立一个新的optimizer来调整
for param_group in optimazer.param_groups:
    param_group['lr'] *= 0.1

In [12]:
# 训练模型
num_epoch = 3
for epoch in range(num_epoch):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1)) # 注意这里要调整y的维度
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    
    print(f'epoch {epoch + 1: d}, loss {l: f}')

epoch  1, loss  0.000113
epoch  2, loss  0.000183
epoch  3, loss  0.000086


In [13]:
# 对比参数
print(true_w, net.linear.weight)
print(true_b, net.linear.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0005, -3.3995]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2003], requires_grad=True)
